# Librerie
caricamento librerie

In [38]:
import pandas as pd

### Carica DataSet
games_march2025_cleaned

In [39]:
dataset = pd.read_csv("DataSet/games_march2025_cleaned.csv")

#dataset

# Data Understanding

In [ ]:
dataset.info()

score_rank troppi null 
metacritic_url non significativo



In [ ]:
dataset.describe()
#statistiche del dataset

* Eliminiamo colonna 'appid', id interno al dataset di steam, non utile alla prediizione
* Eliminiamo colonna della descrizione e raccomandazioni, sono testuali e non utili alla predizione.
    * 'detailed_description' , 'about_the_game', 'short_description', 'reviews'
* Eliminiamo url dell'immagine del gioco
    * 'header_image'
* Eliminiamo colonne testuali per supporto clienti e url vari
    * 'website', 'support_url', 'support_email', 'metacritic_url', 'notes'
* Eliminiamo colonne per trailer video e foto del gioco
    * 'movies' , 'screenshots'
* Elimino i punteggi che hanno troppi valori null
    * 'user_score', 'score_rank'

In [40]:
# drop elimina colonna, per eliminare più colonne contemporaneamente [ , ]
dataset.drop('appid', axis=1, inplace=True)
dataset.drop(['detailed_description','about_the_game', 'short_description', 'reviews', 'header_image'], axis=1, inplace=True)
dataset.drop(['website', 'support_url', 'support_email', 'metacritic_url', 'notes'], axis=1, inplace=True)
dataset.drop(['screenshots','movies'], axis=1, inplace=True)
dataset.drop(['user_score', 'score_rank'], axis=1, inplace=True)

In [ ]:
#dataset.describe(include=['O'])
dataset['user_score'].value_counts()
dataset['metacritic_score'].value_counts()
dataset['score_rank'].value_counts()
dataset['positive'].value_counts()
dataset.info()

In [41]:
# salvo dataset modificato in uno nuovo per poter ripartire da lì
dataset.to_csv('DataSet/games1.csv', index=False)


##### Salvo le modifiche in un nuovo dataset per poterlo utilizzare
* Inizializzare da QUI

In [42]:
#NUOVO DATASET CON MODIFICHE

ds = pd.read_csv("DataSet/games1.csv")

* Creo nuova variabile che include differenza tra positivi e negativi
    * elimino le due colonne di positivi e negativi

In [43]:
ds['absolute_score'] = ds['positive'] - ds['negative']

ds.drop(['positive', 'negative'], axis=1, inplace=True)
ds.to_csv('DataSet/games1.csv', index=False)


### Variabile Target num giocatori che hanno acquistato il gioco
* per decretare successo del gioco

In [44]:
ds['estimated_owners'] = ds['estimated_owners'].apply(lambda x: x.split('-')[0])
ds['estimated_owners'].value_counts()

estimated_owners
0             67738
20000          9683
50000          4552
100000         3005
200000         2469
500000         1038
1000000         574
2000000         373
5000000         108
10000000         42
20000000         24
50000000         10
200000000         1
100000000         1
Name: count, dtype: int64

faccio undersampling e raggruppamento

In [47]:
# dopo il taglio era diventato stringhe
ds['estimated_owners'] = pd.to_numeric(ds['estimated_owners'], errors='coerce')

intervalli = [0, 5000, 50000, 500000, float('inf')]
etichette = ['insuccesso', 'basso_successo', 'medio_successo', 'alto_successo']

ds['success_class'] = pd.cut(
    ds['estimated_owners'],
    bins=intervalli,
    labels=etichette,
    include_lowest=True
)

print(ds[['estimated_owners', 'success_class']].head())

print(ds['success_class'].value_counts())


   estimated_owners  success_class
0         100000000  alto_successo
1          50000000  alto_successo
2         200000000  alto_successo
3          50000000  alto_successo
4          20000000  alto_successo
success_class
insuccesso        67738
basso_successo    14235
medio_successo     6512
alto_successo      1133
Name: count, dtype: int64


In [49]:
from imblearn.under_sampling import RandomUnderSampler

# X = tutte le colonne tranne la target
X = ds.drop(columns=['success_class'])

# y = la target
y = ds['success_class']

# bilancia tutte le classi alla più piccola
undersampler = RandomUnderSampler( sampling_strategy='auto', random_state=42)

# Applichiamo l'undersampling
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Ricombiniamo in un nuovo DataFrame
ds = pd.concat([X_resampled, y_resampled], axis=1)

print("Distribuzione originale:")
print(y.value_counts())

print("\nDistribuzione dopo undersampling:")
print(y_resampled.value_counts())


Distribuzione originale:
success_class
insuccesso        67738
basso_successo    14235
medio_successo     6512
alto_successo      1133
Name: count, dtype: int64

Distribuzione dopo undersampling:
success_class
insuccesso        1133
basso_successo    1133
medio_successo    1133
alto_successo     1133
Name: count, dtype: int64
